## Authenticate and Initialize Earth Engine

In [52]:
!pip install earthengine-api

In [53]:
pip install geemap

Note: you may need to restart the kernel to use updated packages.


In [54]:
import urllib.request
url = "https://raw.githubusercontent.com/nitinmagima/endofseasonmonitoring_madagascar/refs/heads/main/utils.py"
urllib.request.urlretrieve(url, "utils.py")

('utils.py', <http.client.HTTPMessage at 0x25f5c20b860>)

import ee
import geemap
import utils as u
from IPython.display import HTML, Markdown, display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
ee.Authenticate()
ee.Initialize(project="ee-mp4409")

## Initialize Map

In [57]:
import ee
import geemap
import utils as u
from IPython.display import HTML, Markdown, display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [96]:
m = geemap.Map(basemap='Esri.WorldTopoMap')
m.setCenter(-10, 20, 5)
m

Map(center=[20, -10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [98]:
country_name = 'Mauritania'
admin_level = 'level0'

In [99]:
roi = m.user_roi

if roi is None:
    roi = ee.FeatureCollection(f"FAO/GAUL/2015/{admin_level}")
    roi = roi.filter(ee.Filter.eq('ADM0_NAME', country_name))
    styleParams = {
        'fillColor': 'b5ffb4',
        'color': '00909F',
        'width': 1.0,
    }
    styleRoi = roi.style(**styleParams)
    m.addLayer(styleRoi, {}, country_name)
print('ROI type:', roi.getInfo()['type'])


ROI type: FeatureCollection


In [100]:
viirs = ee.ImageCollection('NOAA/VIIRS/001/VNP13A1')
evi_band = 'EVI'
ndvi_band = 'NDVI'
selected_bands = viirs.select([evi_band, ndvi_band])
selected_bands = selected_bands.filterBounds(roi)

In [101]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
spimonthlyvis = {"opacity":1,"bands":["SPI"],"min":-4,"max":4,"palette":["d53e4f","fc8d59","fee08b","ffffbf","e6f598","99d594","3288bd"]}
spi16dayvis = {"opacity":1,"bands":["SPI_16Days"],"min":-4,"max":4,"palette":["d53e4f","fc8d59","fee08b","ffffbf","e6f598","99d594","3288bd"]}

### Set Time Frame

In [103]:
firstimage = ee.Date(ee.List(chirps.get('date_range')).get(0))
latestimage = ee.Date(chirps.limit(1, 'system:time_start',  False).first().get('system:time_start'))

### Set Resolution

In [105]:
resolution = 5550

### Set Time Scale Information For SPI
##### Assess the precipitation anomaly over a specific period. The time scale determines how many months' worth of precipitation you want to include in the calculation of the SPI.

In [107]:
timestep = '1'

### Set Time Shift For VIIRS Related SPI
##### - For example, a shift = -5 would mean that the SPI for a given period is calculated 5 days before the VIIRS data start date and finishes 5 days earlier than the VIIRS data end date.
##### - The days = '16' represents the 16-day period over which the SPI is calculated. This is aligned with the frequency of the VIIRS data (which is also provided on a 16-day interval). This gives you the exact window for which the SPI is calculated.

In [109]:
shift = '0'
days = '16'

### SPI Calculations

In [111]:
thresholdsmonths = ee.Number(12)

In [112]:
timedif = (latestimage.difference(firstimage, 'month')).divide(ee.Number.parse(timestep))

In [113]:
list = ee.List.sequence(0, timedif)

In [114]:
def func_gou(month):
    zero = ee.Number(0)
    delta = (zero.subtract(month)).multiply(ee.Number.parse(timestep))
    # amount：表示你希望推进的数量，可以是一个正数或负数。正数表示将日期推进，负数表示将日期向回推。
    latestdate = latestimage.advance(1, 'day')
    return latestdate.advance(delta, 'month')

timelistdate = list.map(func_gou)

In [115]:
sortedtimelist = timelistdate.sort()

In [130]:
def func_fxo(monthly_sum):
    timestep_num = ee.Number.parse(timestep)
    starttime = ee.Date(monthly_sum).advance(timestep_num.multiply(-1), 'month')
    endtime = ee.Date(monthly_sum)
    filteredCHIRPS = chirps.filterDate(starttime, endtime)
    clippedCHIRPS = filteredCHIRPS.map(lambda clip:clip.clip(roi))
    imageAmount = clippedCHIRPS.size()
    summedCollection = clippedCHIRPS.sum().set({
        'Used_Images': imageAmount,
        'Start_Date': ee.Date(filteredCHIRPS.first().get('system:time_start')),
        'End_Date': ee.Date(filteredCHIRPS.sort('system:time_end', False).first().get('system:time_end')),
        'system:time_start': filteredCHIRPS.first().get('system:time_start'),
        'system:time_end': filteredCHIRPS.sort('system:time_end', False).first().get('system:time_end')
    })
    time = ee.Date(summedCollection.get('system:time_end')).difference(ee.Date(summedCollection.get('system:time_start')), 'month').round()
    
    summedImage = summedCollection.set({
        'Observed_Months': time
    })
    
    return ee.Image(ee.Algorithms.If(time.gte(timestep_num), summedImage))

precipitationsum = ee.ImageCollection.fromImages(ee.List(timelistdate).map(func_fxo))

In [144]:
summedchirpscollection = precipitationsum.map(lambda img: img.copyProperties(chirps))

In [150]:
first_image = summedchirpscollection.first().getInfo()
print(first_image)

{'type': 'Image', 'bands': [{'id': 'precipitation', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'date_range': [347155200000, 1646006400000], 'period_mapping': [347155200000, 1646006400000], 'type_name': 'ImageCollection', 'period': 1, 'keywords': ['chg', 'climate', 'geophysical', 'precipitation', 'ucsb', 'weather'], 'thumb': 'https://mw1.google.com/ges/dd/images/CHIRPS_thumb.png', 'description': '<p>Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS)\nis a 30+ year quasi-global rainfall dataset. CHIRPS incorporates\n0.05° resolution satellite imagery with in-situ station data\nto create gridded rainfall time series for trend analysis and seasonal\ndrought monitoring.</p><p><b>Provider: <a href="https://chc.ucsb.edu/data/chirps">UCSB/CHG</a></b><br><p><b>Resolution</b><br>5566 meters\n</p><p><b>Cadence</b><br>\n  1 day\n</p><p><b>Bands</b><table class="eecat"><tr><th scope="col">